# 模块六实验笔记本：Web 应用开发实战

## 实验目标
本模块我们将掌握 Web 开发的核心技能，实现从“脚本”到“应用”的跨越。

1.  **Lesson 21**: 揭秘 API 与 C/S 架构，搭建“智能餐厅” (FastAPI)。
2.  **Lesson 22**: 前端集成，使用 Fetch API 让网页与 Python 对话。
3.  **Lesson 23**: 案例重构，将 MUD 游戏改造为 Web 版。
4.  **Lesson 24**: 综合工作坊，打造你的 Web 应用作品集。

## 第零步：环境安装 (Setup)

> **⚠️ 重要提示**: 
> 如果是第一次运行此单元格安装依赖，安装完成后，**必须重启内核 (Restart Kernel)**。
> 
> **重启方法**: 点击上方工具栏的 🔄 按钮 (Restart)。

In [ ]:
# 安装 Web 开发核心库
%pip install fastapi uvicorn requests python-multipart qrcode pillow "uvicorn[standard]" -i https://pypi.tuna.tsinghua.edu.cn/simple

---

# Lesson 21: 你的第一个 Web API (智能餐厅)

## 1. 搭建后厨 (Backend)
我们将使用 **FastAPI** 搭建一个提供菜单查询和点餐服务的 API。

运行下面的单元格，它会自动在当前文件夹生成 `server.py` 文件。

In [ ]:
%%writefile server.py
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

app = FastAPI()

# 允许跨域 (CORS) - 解决“同源策略”限制，允许网页访问
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

# 假装这是数据库
menu_data = {
    "红烧肉": {"price": 38, "time": 20},
    "清蒸鱼": {"price": 58, "time": 15},
    "米饭": {"price": 2, "time": 1}
}

class OrderItem(BaseModel):
    dish_name: str
    count: int = 1

@app.get("/")
def root():
    return {"message": "欢迎光临 AI 智能餐厅！"}

@app.get("/menu")
def get_menu():
    """获取菜单"""
    return {"code": 200, "data": menu_data}

@app.get("/menu/{dish_name}")
def get_price(dish_name: str):
    """查询单品价格"""
    if dish_name in menu_data:
        return {"dish": dish_name, "price": menu_data[dish_name]["price"], "time": menu_data[dish_name]["time"]}
    else:
        return {"code": 404, "message": "卖光了"}

@app.post("/order")
def place_order(item: OrderItem):
    """下单接口"""
    if item.dish_name not in menu_data:
        return {"code": 404, "message": f"抱歉，我们没有{item.dish_name}"}
    
    total_price = menu_data[item.dish_name]["price"] * item.count
    # 教学点：这里我们可以定义“业务逻辑”。
    # 逻辑 A (并发): 厨师可以同时做两份，时间不变 
    # 逻辑 B (串行): 厨师一份一份做，时间 * item.count
    wait_time = menu_data[item.dish_name]["time"]
    
    return {
        "code": 200, 
        "message": "下单成功", 
        "total_price": total_price,
        "wait_minutes": wait_time
    }


### 启动服务
请在 VS Code **终端 (Terminal)** 中运行以下命令来启动服务器：
```bash
uvicorn server:app --reload
```
看到 `Uvicorn running on http://127.0.0.1:8000` 即为启动成功。**不要关闭终端。**

## 2. 扮演顾客 (Client)
使用 Python 的 `requests` 库来模拟浏览器发送请求，测试我们的 API。

In [ ]:
import requests

base_url = "http://127.0.0.1:8000"

try:
    # 1. 进店 (GET /)
    print("--- 进店 ---")
    print(requests.get(f"{base_url}/").json())
    
    # 2. 看菜单 (GET /menu)
    print("\n--- 看菜单 ---")
    response = requests.get(f"{base_url}/menu")
    print(response.json())
    
    # 2.5 查单品 (GET /menu/红烧肉)
    print("\n--- 查红烧肉价格 ---")
    print(requests.get(f"{base_url}/menu/红烧肉").json())
    
    # 3. 点菜 (POST /order)
    print("\n--- 点两份红烧肉 ---")
    order_data = {"dish_name": "红烧肉", "count": 2}
    response = requests.post(f"{base_url}/order", json=order_data)
    print(response.json())
    
    # 4. 点个不存在的菜
    print("\n--- 点一份满汉全席 ---")
    bad_order = {"dish_name": "满汉全席"}
    print(requests.post(f"{base_url}/order", json=bad_order).json())

except requests.exceptions.ConnectionError:
    print("❌ 连接失败！请检查您的 uvicorn server 是否正在终端运行。")

---

# Lesson 22: 前端集成 (餐厅前厅)

## 1. 生成交互式网页
现在我们用 HTML + JavaScript 编写一个真正的网页，用 `fetch` 指令去连接上面的 Python 后厨。
运行下面的单元格，自动生成 `index.html`。

In [ ]:
%%writefile index.html
<!DOCTYPE html>
<html lang="zh-CN">
<head>
    <meta charset="UTF-8">
    <title>AI 智能餐厅</title>
    <style>
        body { font-family: 'Segoe UI', sans-serif; padding: 2rem; background-color: #f5f5f5; }
        .container { max-width: 600px; margin: 0 auto; background: white; padding: 2rem; border-radius: 12px; box-shadow: 0 4px 6px rgba(0,0,0,0.1); }
        h1 { text-align: center; color: #333; }
        .menu-item { display: flex; justify-content: space-between; border-bottom: 1px solid #eee; padding: 10px 0; }
        button { background-color: #ff6b6b; color: white; border: none; padding: 5px 15px; border-radius: 20px; cursor: pointer; }
        button:hover { background-color: #ff5252; }
        #status-area { margin-top: 20px; padding: 10px; background: #e3f2fd; border-radius: 8px; display: none; }
    </style>
</head>
<body>
    <div class="container">
        <h1>🍽️ AI 智能餐厅</h1>
        <div id="menu-list">
            <p>正在拉取菜单...</p>
        </div>
        <div id="status-area"></div>
    </div>

    <script>
        const API_BASE = 'http://127.0.0.1:8000';

        // 页面加载时自动获取菜单
        window.onload = async () => {
            try {
                const res = await fetch(`${API_BASE}/menu`);
                const data = await res.json();
                renderMenu(data.data);
            } catch (err) {
                document.getElementById('menu-list').innerHTML = `<p style='color:red'>无法连接后厨 (API)。请检查 Python 是否运行。</p>`;
            }
        };

        function renderMenu(menu) {
            const container = document.getElementById('menu-list');
            container.innerHTML = '';
            for (const [name, info] of Object.entries(menu)) {
                const div = document.createElement('div');
                div.className = 'menu-item';
                div.innerHTML = `
                    <span><b>${name}</b> (¥${info.price}) - ${info.time}分钟</span>
                    <button onclick="order('${name}')">点餐</button>
                `;
                container.appendChild(div);
            }
        }

        async function order(dishName) {
            const statusDiv = document.getElementById('status-area');
            statusDiv.style.display = 'block';
            statusDiv.innerText = `正在下单: ${dishName}...`;
            
            try {
                const res = await fetch(`${API_BASE}/order`, {
                    method: 'POST',
                    headers: {'Content-Type': 'application/json'},
                    body: JSON.stringify({dish_name: dishName, count: 1})
                });
                const result = await res.json();
                statusDiv.innerText = `✅ ${result.message}! 总价: ¥${result.total_price}, 预计等待: ${result.wait_minutes}分钟`;
            } catch (err) {
                statusDiv.innerText = `❌ 下单失败: ${err}`;
            }
        }
    </script>
</body>
</html>


✅ `index.html` 已创建! 请在左侧文件浏览器中双击打开它，或者用浏览器访问它的路径。

---

# Lesson 23: 从脚本到网游 (MUD Game)

## 1. 游戏前端 (Game Frontend)
为了获得最佳体验，我们先为 MUD 游戏生成专用的网页（包含手机扫码、黑色主题等特性）。
**注意：这将覆盖之前的 index.html。**

In [ ]:
%%writefile index.html
<!DOCTYPE html>
<html lang="zh-CN">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">
    <title>MUD Game Web Demo (Lesson 3)</title>
    <style>
        body {
            font-family: 'Consolas', 'Monaco', monospace;
            background-color: #1e1e1e;
            color: #ddd;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
        }

        .container {
            background-color: #252526;
            padding: 1.5rem;
            border-radius: 8px;
            box-shadow: 0 10px 20px rgba(0, 0, 0, 0.5);
            width: 90%;
            max-width: 450px;
            height: 90vh;
            display: flex;
            flex-direction: column;
        }

        h1 {
            text-align: center;
            color: #4CAF50;
            margin: 0 0 1rem 0;
            font-size: 1.5rem;
        }

        .hidden {
            display: none !important;
        }

        /* Login */
        .login-wrap {
            display: flex;
            flex-direction: column;
            height: 100%;
            justify-content: center;
            align-items: center;
            gap: 20px;
        }

        .qr-box {
            text-align: center;
            background: white;
            padding: 10px;
            border-radius: 8px;
        }

        .qr-box img {
            width: 150px;
            height: 150px;
        }

        .qr-box p {
            color: #333;
            margin: 5px 0 0 0;
            font-size: 0.8rem;
            font-weight: bold;
        }

        /* Fix iOS Zoom: Font size must be 16px+ */
        input,
        button {
            padding: 12px;
            border-radius: 4px;
            border: 1px solid #3e3e42;
            background: #333;
            color: white;
            width: 100%;
            box-sizing: border-box;
            font-size: 16px;
        }

        button {
            background-color: #0e639c;
            cursor: pointer;
            font-weight: bold;
            border: none;
            margin-top: 10px;
        }

        button:hover {
            background-color: #1177bb;
        }

        /* Game Layout */
        #game-view {
            display: flex;
            flex-direction: column;
            height: 100%;
            gap: 10px;
        }

        /* Room Info */
        .room-card {
            background: #333;
            padding: 10px;
            border-radius: 4px;
            border-left: 4px solid #4CAF50;
        }

        #room-name {
            font-size: 1.2rem;
            margin: 0;
            color: #fff;
        }

        #room-desc {
            color: #aaa;
            margin: 5px 0;
            font-size: 0.9rem;
        }

        .status-line {
            display: flex;
            gap: 10px;
            font-size: 0.85rem;
            margin-top: 5px;
        }

        .status-item {
            background: #444;
            padding: 2px 6px;
            border-radius: 4px;
        }

        /* Chat Log */
        .log-container {
            flex: 1;
            overflow-y: auto;
            background: #1e1e1e;
            border: 1px solid #333;
            padding: 10px;
            border-radius: 4px;
            font-size: 0.85rem;
        }

        .log-entry {
            margin-bottom: 4px;
            line-height: 1.4;
        }

        .log-entry.sys {
            color: #888;
            font-style: italic;
        }

        .log-entry.chat {
            color: #4ec9b0;
        }

        .log-entry.npc {
            color: #ce9178;
        }

        /* NPC color */

        /* Controls */
        .controls {
            display: grid;
            grid-template-columns: repeat(2, 1fr);
            gap: 8px;
        }

        .exit-btn {
            background-color: #3e3e42;
            margin-top: 0;
        }

        .exit-btn:hover {
            background-color: #505050;
        }

        /* Chat Input */
        .chat-input-area {
            display: flex;
            gap: 5px;
        }

        #chat-msg {
            flex: 1;
        }

        #send-btn {
            width: 60px;
            background-color: #4CAF50;
            margin-top: 0;
        }
    </style>
</head>

<body>

    <div class="container">
        <h1>🗡️ MUD 江湖</h1>

        <!-- Login View -->
        <div id="login-view" class="login-wrap">
            <div class="qr-box">
                <!-- Point to relative path /qrcode -->
                <img src="/qrcode" alt="Scan to Join">
                <p>手机扫码加入</p>
            </div>

            <div style="width: 100%;">
                <input type="text" id="username" placeholder="请输入大侠名号 (如: 郭靖)">
                <button onclick="login()">进入江湖</button>
            </div>
        </div>

        <!-- Game View -->
        <div id="game-view" class="hidden">
            <!-- Top: Room Info -->
            <div class="room-card">
                <h2 id="room-name">Loading...</h2>
                <p id="room-desc">...</p>
                <div class="status-line">
                    <div class="status-item">👤 <span id="others">无</span></div>
                    <div class="status-item">🤖 <span id="npcs">无</span></div>
                    <div class="status-item">📦 <span id="items">无</span></div>
                </div>
            </div>

            <!-- Middle: Chat Log -->
            <div class="log-container" id="message-log">
                <div class="log-entry sys">正在连接服务器...</div>
            </div>

            <!-- Bottom: Controls -->
            <div id="exits" class="controls">
                <!-- Buttons -->
            </div>

            <!-- Chat -->
            <div class="chat-input-area">
                <input type="text" id="chat-msg" placeholder="喊话 (大家都能听见)...">
                <button id="send-btn" onclick="shout()">发送</button>
            </div>
        </div>
    </div>

    <script>
        const API_BASE = "";
        let currentUser = "";
        let pollingInterval = null;

        async function login() {
            const username = document.getElementById('username').value.trim();
            if (!username) return alert("大侠请留名！");

            try {
                const res = await fetch(`${API_BASE}/login`, {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ uid: username })
                });
                const data = await res.json();

                if (data.code === 200) {
                    currentUser = username;
                    document.getElementById('login-view').classList.add('hidden');
                    document.getElementById('game-view').classList.remove('hidden');
                    refreshState();
                    pollingInterval = setInterval(refreshState, 2000);
                }
            } catch (err) {
                alert("无法连接服务器");
            }
        }

        async function refreshState() {
            try {
                const res = await fetch(`${API_BASE}/state?uid=${currentUser}`);
                if (res.status === 404) return window.location.reload();
                const data = await res.json();

                if (data.code === 200) {
                    renderGame(data.data);
                }
            } catch (err) {
                console.error(err);
            }
        }

        function renderGame(state) {
            document.getElementById('room-name').innerText = state.room;
            document.getElementById('room-desc').innerText = state.desc;

            // Statuses
            setText('others', state.others);
            setText('npcs', state.npcs);
            setText('items', state.items);

            // Exits
            const exitsContainer = document.getElementById('exits');
            exitsContainer.innerHTML = '';
            state.exits.forEach(direction => {
                const btn = document.createElement('button');
                btn.className = 'exit-btn';
                btn.innerText = `前往 ${formatDir(direction)}`;
                btn.onclick = () => move(direction);
                exitsContainer.appendChild(btn);
            });

            // Chat Log
            const logContainer = document.getElementById('message-log');

            // Sync Chat (includes NPC talk now)
            logContainer.innerHTML = '';
            state.chat.forEach(msg => {
                let type = 'chat';
                if (msg.startsWith('📢')) type = 'sys';
                else if (msg.startsWith('🤖')) type = 'npc';

                const div = document.createElement('div');
                div.className = `log-entry ${type}`;
                div.innerText = msg;
                logContainer.appendChild(div);
            });

            logContainer.scrollTop = logContainer.scrollHeight;
        }

        function setText(id, list) {
            const el = document.getElementById(id);
            el.innerText = (list && list.length > 0) ? list.join(", ") : "无";
        }

        function formatDir(dir) {
            const maps = { 'north': '北', 'south': '南', 'east': '东', 'west': '西' };
            return maps[dir] || dir;
        }

        async function move(direction) {
            await fetch(`${API_BASE}/move`, {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({ uid: currentUser, target: direction })
            });
            refreshState();
        }

        async function shout() {
            const input = document.getElementById('chat-msg');
            const msg = input.value.trim();
            if (!msg) return;

            await fetch(`${API_BASE}/shout`, {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({ uid: currentUser, target: msg })
            });
            input.value = '';
            refreshState();
        }

        document.getElementById('chat-msg').addEventListener('keypress', function (e) {
            if (e.key === 'Enter') shout();
        });
    </script>
</body>

</html>


## 2. 游戏后端 (Game Server)
这里我们将 Module 2 的 MUD 游戏逻辑封装为 Web API。
核心接口：
- `GET /state?uid=...`: 获取当前房间状态
- `POST /move`: 移动到下一个房间
- `GET /qrcode`: **[NEW]** 获取游戏入口二维码
- `POST /shout`: **[NEW]** 全服喊话

In [ ]:
%%writefile mud_game.py
from fastapi import FastAPI, HTTPException, Body
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import HTMLResponse, Response
from pydantic import BaseModel
from typing import List, Optional, Deque, Dict
from collections import deque
import socket
import qrcode
import io
import pathlib
import random

app = FastAPI()

# --- Configuration: CORS ---
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# --- Data Structures (In-Memory Database) ---

# 1. World Map (Static Graph)
world = {
    "广场": {
        "desc": "这里是桃花岛的中心广场，阳光明媚，微风拂面。",
        "exits": {"north": "客栈", "east": "码头", "west": "铁匠铺", "south": "试剑亭"},
        "npc": [],
        "items": []
    },
    "客栈": {
        "desc": "一家古色古香的客栈，名为'悦来客栈'。店小二正在擦桌子。",
        "exits": {"south": "广场", "east": "厨房"},
        "npc": ["店小二"],
        "items": ["陈年女儿红"]
    },
    "厨房": {
        "desc": "这里烟熏火燎，香气扑鼻，灶台上正炖着一只叫花鸡。",
        "exits": {"west": "客栈"},
        "npc": ["黄蓉"],
        "items": ["叫花鸡", "好逑汤"]
    },
    "码头": {
        "desc": "通往中原的码头，海浪拍打着岸边，几只海鸥在飞翔。",
        "exits": {"west": "广场"},
        "npc": [],
        "items": ["小渔船"]
    },
    "铁匠铺": {
        "desc": "叮叮当当！打铁声不绝于耳，这里热浪滚滚。",
        "exits": {"east": "广场"},
        "npc": ["冯默风"],
        "items": ["大铁锤"]
    },
    "试剑亭": {
        "desc": "一处位于峭壁之上的凉亭，四周云雾缭绕，是习武切磋的好地方。",
        "exits": {"north": "广场", "east": "藏经阁"},
        "npc": ["郭靖"],
        "items": []
    },
    "藏经阁": {
        "desc": "书架上摆满了武功秘籍，空气中弥漫着书香。",
        "exits": {"west": "试剑亭"},
        "npc": ["周伯通"],
        "items": ["九阴真经", "左右互搏术"]
    }
}

# NPC Dialogues
npc_dialogues = {
    "店小二": ["客官，打尖还是住店？", "新到了上好的女儿红，要不尝尝？"],
    "黄蓉": ["靖哥哥去哪了？", "这只叫花鸡火候刚好，快尝尝！", "爹爹又关禁闭了..."],
    "郭靖": ["侠之大者，为国为民。", "蓉儿，你做的菜真好吃。", "降龙十八掌，第一式！"],
    "冯默风": ["打铁也是一种修行。", "这块玄铁可是宝贝。"],
    "周伯通": ["好玩好玩！快来陪我打架！", "我左手画方，右手画圆，你学会了吗？"],
}

# 2. Player Session (Dynamic State)
players = {}

# 3. Global Chat Log (Last 20 messages)
chat_log: Deque[str] = deque(maxlen=20)

# --- Models ---
class PlayerAction(BaseModel):
    uid: str
    target: Optional[str] = None 

# --- Helpers ---
def get_local_ip():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        ip = s.getsockname()[0]
        s.close()
        return ip
    except:
        return "127.0.0.1"

def trigger_npc_event(room_name: str):
    """Event-driven NPC logic"""
    room_info = world.get(room_name)
    if not room_info: return

    npcs = room_info.get("npc", [])
    if npcs: # Always trigger if NPC is present
        npc = random.choice(npcs)
        lines = npc_dialogues.get(npc, ["..."])
        msg = f"🤖 {npc}: {random.choice(lines)}"
        chat_log.append(msg)

# --- Endpoints ---

@app.get("/", response_class=HTMLResponse)
def serve_home():
    return pathlib.Path("index.html").read_text(encoding="utf-8")

@app.get("/qrcode")
def get_qr_image():
    """Generate QR Code for the server URL"""
    ip = get_local_ip()
    url = f"http://{ip}:8000"
    
    qr = qrcode.QRCode(box_size=10, border=4)
    qr.add_data(url)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")
    
    buf = io.BytesIO()
    img.save(buf, format="PNG")
    buf.seek(0)
    return Response(content=buf.getvalue(), media_type="image/png")

@app.post("/login")
def login(action: PlayerAction):
    if action.uid not in players:
        players[action.uid] = {"loc": "广场"}
        chat_log.append(f"📢 系统: 欢迎少侠 {action.uid} 踏入江湖！")
        trigger_npc_event("广场") # Trigger NPC
        return {"code": 200, "message": f"Welcome, {action.uid}!"}
    else:
        return {"code": 200, "message": f"Welcome back, {action.uid}!"}

@app.get("/state")
def get_state(uid: str, last_idx: int = 0):
    if uid not in players:
        raise HTTPException(status_code=404, detail="Player not found")
    
    current_room_name = players[uid]["loc"]
    room_info = world[current_room_name]
    
    # 1. Get Players
    others = [
        name for name, data in players.items() 
        if data["loc"] == current_room_name and name != uid
    ]
    
    # 2. Get NPCs (Static info only)
    current_npcs = room_info.get("npc", [])

    return {
        "code": 200,
        "data": {
            "room": current_room_name,
            "desc": room_info["desc"],
            "exits": list(room_info["exits"].keys()),
            "items": room_info.get("items", []),
            "npcs": current_npcs,
            "others": others,
            "chat": list(chat_log)
        }
    }

@app.post("/move")
def move(action: PlayerAction):
    if action.uid not in players:
        raise HTTPException(status_code=404, detail="Player not found")
    
    current_room_name = players[action.uid]["loc"]
    room_exits = world[current_room_name]["exits"]
    direction = action.target
    
    if direction in room_exits:
        new_room = room_exits[direction]
        players[action.uid]["loc"] = new_room
        trigger_npc_event(new_room) # Trigger NPC
        return {"code": 200, "message": f"你前往了 {new_room}"}
    else:
        return {"code": 400, "message": "那边没有路！"}

@app.post("/shout")
def shout(action: PlayerAction):
    """Global Chat"""
    if action.uid not in players:
        raise HTTPException(status_code=404, detail="Player not found")
    
    msg = action.target
    if msg:
        chat_log.append(f"[{action.uid}]: {msg}")
        return {"code": 200, "message": "Sent"}
    return {"code": 400, "message": "Empty message"}


✅ `mud_game.py` 已创建! 
**注意**: 在终端里按 `Ctrl+C` 停止之前的 server，然后运行: 
```bash
uvicorn mud_game:app --reload
```
现在，用手机扫描屏幕上出现的二维码（或者访问 http://你的IP:8000/qrcode），即可加入游戏！

---

# Lesson 24: 综合工作坊 (Workshop)

## Track B: 效率工具箱 (文件上传)
这是 Track B 赛道的核心代码模板。本模块我们专注于**机制**（如何让 Python 从浏览器接收文件），这是后续接入 AI 处理文档的前提。

In [ ]:
%%writefile upload_app.py
from fastapi import FastAPI, UploadFile

app = FastAPI()

@app.post("/upload")
async def upload_file(file: UploadFile):
    # 读取文件内容
    content = await file.read()
    
    # 这里可以添加转换为 Markdown 的 AI 逻辑
    # ...
    
    return {
        "filename": file.filename,
        "size": len(content),
        "content_type": file.content_type,
        "message": "文件接收成功！准备进行 AI 处理..."
    }
